## 쇼핑몰 크롤링 DB에 저장

데이터베이스 생성


In [2]:
pip install emoji

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import emoji

In [4]:
def conn(d_name):
    import pymysql
    host_name='localhost'
    host_port=3306
    username='root'
    password='admin'
    database_name='beauty_shop'

    db=pymysql.connect(
        host=host_name,
        port=host_port,
        user=username,
        password=password,
        database=database_name,
        charset='utf8'
    )
    return db

In [5]:
db=conn('beauty_shop')
cursor=db.cursor()

In [6]:
sql="SHOW DATABASES"
cursor.execute(sql)
result=cursor.fetchall()
result

(('beauty_shop',),
 ('ecommerce',),
 ('employees',),
 ('information_schema',),
 ('mysql',),
 ('performance_schema',),
 ('sakila',),
 ('sqldb',),
 ('student_mgmt',),
 ('sys',),
 ('tabledb',),
 ('world',))

In [7]:
sql='USE beauty_shop'
cursor.execute(sql)

0

In [8]:
sql="SELECT DATABASE()"
cursor.execute(sql)
result=cursor.fetchone()
result

('beauty_shop',)

In [9]:
sql='''
CREATE TABLE IF NOT EXISTS product (
PRODUCT_CODE int AUTO_INCREMENT NOT NULL,
TITLE VARCHAR(200) NOT NULL,
ORI_PRICE VARCHAR(100),
DISCOUNT_PRICE VARCHAR(100),
link VARCHAR(200),
PRIMARY KEY(PRODUCT_CODE)
);
'''

cursor.execute(sql)
db.commit()

In [10]:
sql="SHOW TABLES"

cursor.execute(sql)
result=cursor.fetchall()
result

(('product',),)

In [11]:
sql="DESC product"
cursor.execute(sql)
result=cursor.fetchall()
result

'''
👉 product 테이블의 구조(컬럼 정보)를 보여줘 라는 뜻이지
👉 정렬(ORDER BY) 과는 아무 관련이 없어
✅ 테이블을 만들 때 정의된 컬럼 순서 그대로
(CREATE TABLE 할 때 적은 순서)
'''

'\n👉 product 테이블의 구조(컬럼 정보)를 보여줘 라는 뜻이지\n👉 정렬(ORDER BY) 과는 아무 관련이 없어\n✅ 테이블을 만들 때 정의된 컬럼 순서 그대로\n(CREATE TABLE 할 때 적은 순서)\n'

In [12]:
url="http://jolse.com/category/toners-mists/1019/"
html=urlopen(url)
htmls=html.read()
bs_obj=BeautifulSoup(htmls,"html.parser")

In [13]:
def get_product_info(box):
    strong_tag=box.find("strong",{"class":"name"})
    span=strong_tag.text.split(":",1)[1]

    a=strong_tag.find("a")
    sub_link="https://jolse.com" + a['href']
    sub_link=emoji.replace_emoji(sub_link,'')
    price_ul=box.find("ul")

    old_price=price_ul.select("li")[0]
    old_price=old_price.select("span")[1].text
    old_price=old_price.split(" ")[1]

    dis_price=price_ul.select("li")[1]
    dis_price=dis_price.select("span")[1].text
    dis_price=dis_price.split(" ")[1]

    title=span.replace("'","''")
    title=emoji.replace_emoji(title,'')
    if dis_price=='':
        dis_price='0.0'
    
    return{"prd_name":title,"price":old_price,"sale_price":dis_price,"sub_link":sub_link}


In [14]:
a=bs_obj.find("strong",{"class":"name"})
print(a)


<strong class="name"><a class="" href="/product/b-lab-essence-toner-pad-10ea25g/89277/category/1019/display/1/"><span class="title displaynone"><span class="" style="font-size:14px;color:#555555;font-weight:bold;">Product Name</span> :</span> <span class="" style="font-size:14px;color:#555555;font-weight:bold;">B : Lab Essence Toner Pad 10ea/25g</span></a></strong>


In [15]:
def save_data(prd_info):
    sql="INSERT INTO product (title,ori_price,discount_price,link) values('"\
    + prd_info["prd_name"] \
    + "',"\
    + prd_info['price'] \
    +","\
    + prd_info['sale_price'] \
    +",'"\
    + prd_info['sub_link']\
    + "');"
    cursor.execute(sql)

In [16]:
def get_page_products(url):
    url=url
    html=urlopen(url)
    htmls=html.read()
    bs_obj=BeautifulSoup(htmls,"html.parser")
    ul=bs_obj.find("ul",{"class":"prdList grid5"})

    prd_boxes=ul.findAll("div",{"class":"description"})

    for box in prd_boxes:
        prd=get_product_info(box)
        
        print(prd)

        save_data(prd)

In [17]:
from tqdm import tqdm
url="http://jolse.com/category/toners-mists/1019/?page="

last=2

for i in tqdm(range(1,last+1)):
    urlfin=url+str(i)
    get_page_products(urlfin)
    


 50%|█████     | 1/2 [00:00<00:00,  1.27it/s]

{'prd_name': ' B : Lab Essence Toner Pad 10ea/25g', 'price': '5.00', 'sale_price': '1.99', 'sub_link': 'https://jolse.com/product/b-lab-essence-toner-pad-10ea25g/89277/category/1019/display/1/'}
{'prd_name': ' EQQUALBERRY Swimming Pool Toner 300ml', 'price': '22.00', 'sale_price': '15.40', 'sub_link': 'https://jolse.com/product/eqqualberry-swimming-pool-toner-300ml/87123/category/1019/display/1/'}
{'prd_name': ' celimax Jiwoogae Heartleaf BHA Peeling Pad 60pcs (22AD)', 'price': '17.50', 'sale_price': '13.12', 'sub_link': 'https://jolse.com/product/celimax-jiwoogae-heartleaf-bha-peeling-pad-60pcs-22ad/55940/category/1019/display/1/'}
{'prd_name': ' SKIN1004 Madagascar Centella Probio-Cica Essense Toner 210ml', 'price': '24.00', 'sale_price': '14.49', 'sub_link': 'https://jolse.com/product/skin1004-madagascar-centella-probio-cica-essense-toner-210ml/68504/category/1019/display/1/'}
{'prd_name': ' Dr.Althea 345 Relief Cream Mist 100ml', 'price': '19.90', 'sale_price': '13.93', 'sub_link':

100%|██████████| 2/2 [00:01<00:00,  1.08it/s]

{'prd_name': ' Anua HEARTLEAF 77% SOOTHING TONER 40ml', 'price': '8.00', 'sale_price': '6.80', 'sub_link': 'https://jolse.com/product/anua-heartleaf-77-soothing-toner-40ml/68871/category/1019/display/1/'}
{'prd_name': ' Isntree Green tea Fresh Toner 200ml', 'price': '19.90', 'sale_price': '15.92', 'sub_link': 'https://jolse.com/product/isntree-green-tea-fresh-toner-200ml/15753/category/1019/display/1/'}
{'prd_name': ' COSRX CENTELLA WATER ALCOHOL FREE TONER 150ml', 'price': '17.00', 'sale_price': '11.90', 'sub_link': 'https://jolse.com/product/cosrx-centella-water-alcohol-free-toner-150ml/3367/category/1019/display/1/'}
{'prd_name': ' HAYEJIN RiceFila Oil Toner 150ml', 'price': '30.00', 'sale_price': '25.50', 'sub_link': 'https://jolse.com/product/hayejin-ricefila-oil-toner-150ml/82560/category/1019/display/1/'}
{'prd_name': ' mixsoon Galactomyces Toner 300ml', 'price': '25.00', 'sale_price': '16.25', 'sub_link': 'https://jolse.com/product/mixsoon-galactomyces-toner-300ml/55618/categor

In [18]:
db.commit()

In [19]:
sql="SELECT * FROM product"
cursor.execute(sql)
result=cursor.fetchall()
result[:5]

((1,
  ' B : Lab Essence Toner Pad 10ea/25g',
  '5.00',
  '1.99',
  'https://jolse.com/product/b-lab-essence-toner-pad-10ea25g/89277/category/1019/display/1/'),
 (2,
  ' EQQUALBERRY Swimming Pool Toner 300ml',
  '22.00',
  '15.40',
  'https://jolse.com/product/eqqualberry-swimming-pool-toner-300ml/87123/category/1019/display/1/'),
 (3,
  ' celimax Jiwoogae Heartleaf BHA Peeling Pad 60pcs (22AD)',
  '17.50',
  '13.12',
  'https://jolse.com/product/celimax-jiwoogae-heartleaf-bha-peeling-pad-60pcs-22ad/55940/category/1019/display/1/'),
 (4,
  ' SKIN1004 Madagascar Centella Probio-Cica Essense Toner 210ml',
  '24.00',
  '14.49',
  'https://jolse.com/product/skin1004-madagascar-centella-probio-cica-essense-toner-210ml/68504/category/1019/display/1/'),
 (5,
  ' Dr.Althea 345 Relief Cream Mist 100ml',
  '19.90',
  '13.93',
  'https://jolse.com/product/dralthea-345-relief-cream-mist-100ml/92356/category/1019/display/1/'))

In [20]:
db.close()

## db테이블에 저장된 데이터 df로 가져오기

In [21]:
db=conn("beauty_shop")

sql="SELECT * FROM product"

df=pd.read_sql(sql,db)
df

C:\Users\User\AppData\Local\Temp\ipykernel_27208\3013618092.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(sql,db)


,PRODUCT_CODE,TITLE,ORI_PRICE,DISCOUNT_PRICE,link
0,1,B : Lab Essence Toner Pad 10ea/25g,5.00,1.99,https://jolse.com/product/b-lab-essence-toner-...
1,2,EQQUALBERRY Swimming Pool Toner 300ml,22.00,15.40,https://jolse.com/product/eqqualberry-swimming...
2,3,celimax Jiwoogae Heartleaf BHA Peeling Pad 60...,17.50,13.12,https://jolse.com/product/celimax-jiwoogae-hea...
3,4,SKIN1004 Madagascar Centella Probio-Cica Esse...,24.00,14.49,https://jolse.com/product/skin1004-madagascar-...
4,5,Dr.Althea 345 Relief Cream Mist 100ml,19.90,13.93,https://jolse.com/product/dralthea-345-relief-...
...,...,...,...,...,...
75,76,numbuzin No.1 Centella Re-Leaf Green Toner Pa...,24.00,16.80,https://jolse.com/product/numbuzin-no1-centell...
76,77,Isntree Hyaluronic Acid Toner Plus 200ml,22.20,17.76,https://jolse.com/product/isntree-hyaluronic-a...
77,78,APLB Glutathione Niacinamide Mist Essence 105ml,16.40,10.17,https://jolse.com/product/aplb-glutathione-nia...
78,79,Anua Azelaic 10 Hyaluron Redness Soothing Pad...,27.00,22.95,https://jolse.com/product/anua-azelaic-10-hyal...
